In [1]:
# Importing libraries;
from scripts import bronze, silver, gold, utils

# Setup logger;
logger = utils.setup_logging()

# Set up lakehouse architecture;
logger = utils.setup_lakehouse(logger)

2025-03-26 21:13:15,957 - INFO - Configuração de logging concluída.
2025-03-26 21:13:15,959 - INFO - Diretórios da arquitetura Lakehouse verificadas com sucesso.


In [2]:
# BRONZE: Transform CSV to PARQUET file;
df_bronze = bronze.etl_bronze(logger)

2025-03-26 21:13:16,035 - INFO - Arquivo CSV carregado com sucesso de data-source/Emissão_CO2_por_países.csv. Registros: 5677
2025-03-26 21:13:16,084 - INFO - Arquivo Parquet salvo na Bronze Zone: data-lakehouse/bronze/Emissão_CO2_por_países.parquet


In [4]:
# SILVER: Clean Dataset;
df_silver = silver.etl_silver(logger)

2025-03-26 21:13:21,612 - INFO - Arquivo carregado. Registros: 5677
2025-03-26 21:13:21,701 - INFO - Arquivo salvo na Silver Zone: data-lakehouse/silver/Emissão_CO2_por_países.parquet


In [5]:
# GOLD: Apply Bussiness Intelligence;
df_gold = gold.etl_gold(logger)

2025-03-26 21:13:46,380 - INFO - Arquivo carregado da Silver Zone. Registros: 5677
2025-03-26 21:13:46,398 - INFO - Resultados de BI salvos com sucesso na Gold Zone!


In [8]:
import pandas as pd

# Lendo os arquivos Parquet
co2_by_region = pd.read_parquet('data-lakehouse/gold/co2_by_region.parquet')
top_countries_co2 = pd.read_parquet('data-lakehouse/gold/top_countries_co2.parquet')

# Verificando as primeiras linhas dos dataframes
print(co2_by_region.head())
print(top_countries_co2.head())


     region  kilotons_of_co2
0      Asia     3.705193e+08
1  Americas     2.195244e+08
2    Europe     1.839814e+08
3    Africa     2.884081e+07
4   Oceania     1.151804e+07
         country  kilotons_of_co2
0          China     1.807228e+08
1  United States     1.583179e+08
2         Russia     5.002867e+07
3          India     3.988273e+07
4          Japan     3.507659e+07


In [14]:
import dash
from dash import dcc, html
import plotly.express as px
import pandas as pd

# Inicializando o app Dash
app = dash.Dash(__name__)

# Selecionando os 10 países com as maiores emissões de CO2
top_countries_co2 = top_countries_co2.nlargest(10, 'kilotons_of_co2')

# Criando gráficos interativos com Plotly
fig1 = px.bar(co2_by_region, x='region', y='kilotons_of_co2', title="Emissões de CO2 por Região")
fig2 = px.bar(top_countries_co2, x='country', y='kilotons_of_co2', title="Top 10 Países por Emissões de CO2")

# Definindo o layout do dashboard
app.layout = html.Div(children=[
    html.H1("Dashboard de Emissões de CO2", style={'textAlign': 'center'}),
    dcc.Graph(id='grafico1', figure=fig1),
    dcc.Graph(id='grafico2', figure=fig2),
])

# Rodando o servidor
if __name__ == '__main__':
    app.run(debug=True)